In [1]:
import json, os
from openai import OpenAI

source_data_path = "../data/raw_data/"

filename = "summary_zho_Territorial_disputes_in_the_South_China_Sea_100.json"
# filename = "summary_eng_Territorial_disputes_in_the_South_China_Sea_100.json"
source_filename = source_data_path + filename

des_data_path = ""
des_filename = des_data_path + "narrative_QnA_" + filename

# Load articles
with open(source_filename, "r", encoding="utf-8") as f:
    articles = json.load(f)

# Load questions
with open("formated_output_questions.json", "r", encoding="utf-8") as f:
    questions = json.load(f)

# Set API key
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [2]:
def get_answers(article, questions, model):
    article_title = article.get("title")
    article_content = article.get("body")
    print(f"Proceeding article {article_title}")
    # answers = {}
    # for q_id, question in questions.items():

    pre = "You are an AI trained to understand articles and generate concise answers to multiple-choice questions based on the content. \
        Please read the following article carefully. After reading, answer ALL the questions listed below. \
            Your answers must be in capital letters and formatted as a single string, where each question number is followed by its corresponding answer letter. \
                Separate each question-answer pair with a semicolon. \
                    Example format: 1A;2B;3C;4D;... \n\n"

    prompt = pre + f"Article Content:\n{article_content}\n\n Questions: {questions}\n"
    try:
        response = client.chat.completions.create(
            model=model, messages=[{"role": "user", "content": prompt}], temperature=0
        )
        # content = response.choices[0].message.content
        # answers[q_id] = content
    except Exception as e:
        print(f"Error: {e}")
        # answers[q_id] = "Sorry, error from GPT."
    return response

In [3]:
def convert_string_to_dict(answer_string):
    if answer_string == "":
        return {}
    if answer_string[-1] == ";":
        answer_string = answer_string[:-1]
    return {item[:-1]: item[-1] for item in answer_string.split(";")}


results = []
iteration = 0
max_iteration = len(articles)

for article in articles:
    # article_content = article['body']

    answers = get_answers(article, questions, model="gpt-4o")

    pure_answers = answers.choices[0].message.content
    print(pure_answers)

    # convert string to dictionary 1A;2B;3C;44D to {'1': 'A', '2': 'B', '3': 'C', '44': 'D'}
    processed_answers = convert_string_to_dict(pure_answers)
    article["answers"] = processed_answers

    results.append(article)
    iteration += 1
    print(f"Processed {iteration} out of {max_iteration}")


# Save results to a JSON file
with open(des_filename, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

Proceeding article 中共"战狼"少将引拜登讲话 被指张冠李戴 - 大纪元
1A;2A;3A;4A;5D;6C;7D;8B;9A;10A;11E;12E;13E
Processed 1 out of 100
Proceeding article 槍指中國海警？ 北京要求菲律賓莫再製造事端 | 兩岸 | 中央社 CNA
1C;2C;3B;4B;5A;6E;7A;8A;9B;10A;11A;12E;13A
Processed 2 out of 100
Proceeding article 槍指中國海警？ 北京要求菲律賓莫再製造事端 | 聯合新聞網
1C;2C;3B;4B;5A;6E;7A;8A;9B;10A;11A;12E;13A
Processed 3 out of 100
Proceeding article 槍指中國海警？ 北京要求菲律賓莫再製造事端
1C;2C;3B;4B;5A;6E;7A;8A;9B;10A;11A;12E;13A
Processed 4 out of 100
Proceeding article 中国驻韩大使馆称"韩国对台湾和南中国海问题说三道四"
1E;2E;3E;4E;5E;6E;7E;8E;9E;10A;11E;12E;13E
Processed 5 out of 100
Proceeding article 与泽连斯基见过一面后，马科斯被警告：别让菲律宾成为"乌克兰"
1E;2E;3E;4C;5E;6B;7E;8C;9C;10A;11E;12E;13E
Processed 6 out of 100
Proceeding article 香格里拉对话｜现场观察：一场美西方构筑信息茧房的盛宴_澎湃防务_澎湃新闻-The Paper
1A;2A;3A;4A;5D;6C;7D;8A;9B;10C;11A;12E;13E
Processed 7 out of 100
Proceeding article 菲律宾"既要又要"：我们还可以跟中国做生意
1C;2C;3B;4B;5A;6E;7A;8A;9D;10A;11A;12E;13A
Processed 8 out of 100
Proceeding article 中方忍耐已到极限，无差别拦截仁爱礁菲律宾船只，补给全倒海里
1C;2C;3B;4B;5A;6E;7A;8A;9A;1